In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split  
from keras.utils import to_categorical
import re
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.resetwarnings()

In [2]:
#read data
df=pd.read_csv('Twitter_Data.csv')

In [3]:
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [4]:
# Removing category=0 (neutrality)
df=df[df.category!=0]

In [5]:
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
8,with upcoming election india saga going import...,1.0
...,...,...
162972,engine growth modi unveils indias first 12000 ...,1.0
162973,modi promised 2014 lok sabha elections that be...,1.0
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0


In [6]:
df['clean_text']=df['clean_text'].apply(lambda x: str(x).lower())
df['clean_text']=df['clean_text'].apply((lambda x:re.sub('[^A-Za-z0-9\s]','',x)))

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
C:\Users\Farah\AppData\Local\Temp\ipykernel_11960\3999792764.py:2: DeprecationWarning: invalid escape sequence \s
  df['clean_text']=df['clean_text'].apply((lambda x:re.sub('[^A-Za-z0-9\s]','',x)))
C:\Users\Farah\AppData\Local\Temp\ipykernel_11960\3999792764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text']=df['clean_text'].apply(lambda x: str(x).lower())
C:\Users\Farah\AppData\Local\Temp\ipykernel_11960\3999792764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [7]:
df['category'].value_counts()

 1.0    72250
-1.0    35510
Name: category, dtype: int64

In [8]:
max_feature=2000
tokenizer=Tokenizer(num_words=max_feature,split=' ')
tokenizer.fit_on_texts(df['clean_text'].values)
x=tokenizer.texts_to_sequences(df['clean_text'].values)

In [9]:
x=pad_sequences(x)
len(x[1])

47

In [10]:
y=pd.get_dummies(df['category']).values
y.shape

(107767, 2)

# Model Training

In [11]:
embed_dim=128
lstm_out=196
model=Sequential()
model.add(Embedding(max_feature,embed_dim,input_length=x.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 47, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
embed_dim=128
lstm_out=196
model=Sequential()
model.add(Embedding(max_feature,embed_dim,input_length=x.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)
batch_size=32
model.fit(x_train,y_train,epochs=7,batch_size=batch_size,verbose=1)

Epoch 1/7
2257/2257 [==============================] - 918s 403ms/step - loss: 0.2978 - accuracy: 0.8727
Epoch 2/7
2257/2257 [==============================] - 770s 341ms/step - loss: 0.2017 - accuracy: 0.9229
Epoch 3/7
2257/2257 [==============================] - 568s 252ms/step - loss: 0.1902 - accuracy: 0.9286
Epoch 4/7
2257/2257 [==============================] - 569s 252ms/step - loss: 0.1794 - accuracy: 0.9322
Epoch 5/7
2257/2257 [==============================] - 550s 244ms/step - loss: 0.1688 - accuracy: 0.9367
Epoch 6/7
2257/2257 [==============================] - 547s 242ms/step - loss: 0.1598 - accuracy: 0.9412
Epoch 7/7
2257/2257 [==============================] - 554s 246ms/step - loss: 0.1521 - accuracy: 0.9438


# Evaluating model

In [13]:
batch_size=32
validation_size=1500
x_val=x_test[-validation_size:]
y_val=y_test[-validation_size:]
x_test=x_test[:-validation_size]
y_test=y_test[:-validation_size]
score,acc=model.evaluate(x_test,y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)

1065/1065 - 54s - loss: 0.2082 - accuracy: 0.9267 - 54s/epoch - 51ms/step
0.2081538736820221
0.9267261624336243


# Testing model

In [34]:
ex1=["I absolutely loved the new movie! The plot was engaging, the characters were well-developed, and the ending was so satisfying. Can't wait to watch it again!"]
ex1=tokenizer.texts_to_sequences(ex1)
ex1=pad_sequences(ex1,maxlen=47,dtype='int32',value=0)
senti1=model.predict(ex1,batch_size=1,verbose=2)[0]


1/1 - 0s - 53ms/epoch - 53ms/step


In [35]:
ex2=["I was extremely disappointed with the service at the restaurant. The staff was rude, the food took forever to arrive, and when it did, it was cold. I won't be going back "]
ex2=tokenizer.texts_to_sequences(ex2)
ex2=pad_sequences(ex2,maxlen=47,dtype='int32',value=0)
senti2=model.predict(ex2,batch_size=1,verbose=2)[0]

1/1 - 0s - 45ms/epoch - 45ms/step


In [37]:
import numpy as np
if(np.argmax(senti1)==0):
    print('negatif')
else:
    print('positif')

positif


In [38]:
import numpy as np
if(np.argmax(senti2)==0):
    print('negatif')
else:
    print('positif')

negatif
